# Import de librerias

In [1]:
import http.client
import json
import pandas as pd
from dotenv import load_dotenv
import os

# Conexion con la API

In [5]:
# Carga las variables de entorno
load_dotenv()

# Obtén la API key
api_key = os.getenv('API_KEY')

conn = http.client.HTTPSConnection("idealista7.p.rapidapi.com")

headers = {
    'x-rapidapi-key': api_key,
    'x-rapidapi-host': "idealista7.p.rapidapi.com"
}

In [6]:
df_vivienda = pd.DataFrame()
df_localizacion = pd.DataFrame()
df_descripcion = pd.DataFrame()


for i in range(1,40):
    conn.request("GET", f"/listhomes?order=relevance&operation=sale&locationId=0-EU-ES-28-07-001-079&locationName=Madrid&numPage={i}&maxItems=40&location=es&locale=en", headers=headers)

    res = conn.getresponse()
    json_data = res.read()

    if json_data:
        try:
            data_dict = json.loads(json_data)
        except json.JSONDecodeError as e:
            print("Error al decodificar JSON:", e)
    else:
        print("La respuesta está vacía.")
        break
        
    
    # Navegamos sobre el JSON
    if 'elementList' in data_dict:  # Reemplaza 'listings' con la clave correcta si es diferente
        data_list = data_dict['elementList']  # Accede a la lista de propiedades
    else:
        print("La clave 'listings' no se encuentra en el JSON.")
        data_list = []

    # Extraer solo los campos seleccionados y convertir en un DataFrame
    selected_fields = [
        {
            "propertyCode": item.get("propertyCode"),
            "url": item.get("url"),
            "address": item.get("address"),
            "size": item.get("size"),
            "floor": item.get("floor"),
            "province": item.get("province"),
            "municipality": item.get("municipality"),
            "district": item.get("district"),
            "price": item.get("price"),
            "rooms": item.get("rooms"),
            "hasLift": item.get("hasLift"),
            "hasParking": item.get("parkingSpace", {}).get("hasParkingSpace"),
            "hasTerrace": item.get("features", {}).get("hasTerrace"),
            "hasSwimmingPool": item.get("features", {}).get("hasSwimmingPool"),
            "hasAirConditioning": item.get("features", {}).get("hasAirConditioning"),
            "hasGarden": item.get("features", {}).get("hasGarden"),
            "bathrooms": item.get("bathrooms"),
            "latitude": item.get("latitude"),
            "longitude": item.get("longitude"),
            "description": item.get("description"),
        }
        for item in data_list
    ]
    
    # Convertir la lista de diccionarios en un DataFrame
    df = pd.DataFrame(selected_fields)
    
    df_vivienda_aux = df.drop(['latitude', 'longitude', 'description'], axis=1)
    df_localizacion_aux = df[['propertyCode','latitude', 'longitude']]
    df_descripcion_aux = df[['propertyCode', 'description']]
    
    df_vivienda = pd.concat([df_vivienda,df_vivienda_aux], axis=0)
    df_localizacion = pd.concat([df_localizacion, df_localizacion_aux], axis=0)
    df_descripcion = pd.concat([df_descripcion, df_descripcion_aux], axis=0)
        

# Carga de datos

In [9]:
csv_vivienda = df_vivienda.to_csv('../data/structured/vivienda.csv', index=False)
csv_localizacion = df_localizacion.to_csv('../data/structured/location.csv', index=False)
json_description = df_descripcion.to_json('../data/unstructured/archivo.json', orient='records', indent=4)

In [10]:
df_vivienda.describe()

,size,price,rooms,bathrooms
count,1560.000000,1.560000e+03,1560.000000,1560.000000
mean,208.676282,1.311048e+06,3.332051,2.764103
std,201.257569,1.190102e+06,1.665019,1.665634
min,18.000000,3.500000e+04,0.000000,1.000000
25%,98.000000,5.350000e+05,2.000000,2.000000
50%,145.000000,9.800000e+05,3.000000,2.000000
75%,246.250000,1.690000e+06,4.000000,3.000000
max,1768.000000,1.995000e+07,13.000000,13.000000
